In [3]:
import modelbit
mb = modelbit.login()

## REAL TIME MODEL DEPLOYMENT

In [7]:
import warnings
warnings.filterwarnings('ignore')
import torch
from torch import nn
from torchvision import transforms, models
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import base64

class Model(nn.Module):
    def __init__(self, num_classes=2, latent_dim=2048, lstm_layers=1, hidden_dim=2048, bidirectional=False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained=True)  # Residual Network CNN
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim, hidden_dim, lstm_layers, bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = self.model(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return self.dp(self.linear1(x))

def deepfake_detection(frame_base64):
    frame_data = base64.b64decode(frame_base64)
    frame = Image.open(BytesIO(frame_data))

    im_size = 224
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    transform = transforms.Compose([
        transforms.Resize((im_size, im_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    frame = transform(frame).unsqueeze(0)
    
    # Load the TorchScript model
    model_path = "traced_model.pt"
    model = torch.jit.load(model_path)
    
    model.eval()

    with torch.no_grad():
        frame_tensor = frame
        logits = model(frame_tensor)
        sm = nn.Softmax(dim=1)
        probabilities = sm(logits)
        confidence, prediction = torch.max(probabilities, dim=1)    

    if prediction == 1:
        result = {
            "status": "real",
            "confidence": confidence.item() * 100
        }
    else:
        result = {
            "status": "fake",
            "confidence": confidence.item() * 100
        }
    
    return result

In [8]:
mb.deploy(deepfake_detection, extra_files={"/Users/karthiksagar/DeepFake-Detection/saved_best_model/traced_model.pt" : "traced_model.pt"})

## UPLOAD VIDEO DEPLOYMENT

In [13]:
import cv2
import torch
import torchvision.transforms as transforms
from PIL import Image
import os

# Model class
class Model(nn.Module):
    def __init__(self, num_classes=2, latent_dim=2048, lstm_layers=1, hidden_dim=2048, bidirectional=False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained=True)  # Residual Network CNN
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim, hidden_dim, lstm_layers, bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = self.model(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return self.dp(self.linear1(x))

# Preprocessing transforms (same as during training)
im_size = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
transform = transforms.Compose([
    transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

def extract_frames(video_path, frame_count=20):
    """
    Extract frames from the video at regular intervals.
    
    Args:
    - video_path: Path to the video file.
    - frame_count: Number of frames to extract.
    
    Returns:
    - A list of extracted frames (PIL Images).
    """
    vidcap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = total_frames // frame_count
    
    success, image = vidcap.read()
    count = 0
    
    while success and len(frames) < frame_count:
        if count % frame_interval == 0:
            # Convert to PIL Image and append
            frames.append(Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)))
        success, image = vidcap.read()
        count += 1
    
    vidcap.release()
    return frames

def preprocess_frames(frames):

    preprocessed_frames = [transform(frame) for frame in frames]
    return torch.stack(preprocessed_frames).unsqueeze(0)  # Add batch dimension

def predict_video(video_path):

    # Load the model
    model_path = 'traced_model.pt'
    model = torch.jit.load(model_path)
    model.eval()
    
    # Extract and preprocess frames from the video
    frames = extract_frames(video_path, frame_count=20)
    frame_tensor = preprocess_frames(frames)
    
    # Make prediction on the frame sequence
    with torch.no_grad():
        logits = model(frame_tensor)
        sm = torch.nn.Softmax(dim=1)
        probabilities = sm(logits)
        confidence, prediction = torch.max(probabilities, dim=1)
    
    result = {
        "status": "real" if prediction.item() == 1 else "fake",
        "confidence": confidence.item() * 100
    }

    return result


In [14]:
mb.deploy(predict_video, extra_files={"/Users/karthiksagar/DeepFake-Detection/saved_best_model/traced_model.pt" : "traced_model.pt"})